## Process the Customers Data 
1. Ingest the data into the data lakehouse - bronze_customers
1. Perform data quality checks and transform the data as required - silver_customers_clean
1. Apply changes to the Customers data - silver_customers

In [0]:
-- 1.

CREATE OR REFRESH STREAMING TABLE bronze_customers
COMMENT 'Raw customers data ingested from the source system operational data'
TBLPROPERTIES ('quality' = 'bronze')
AS 
SELECT *,
        _metadata.file_path AS file_path,
        CURRENT_TIMESTAMP AS ingestion_timestamp
FROM cloud_files(
  '/Volumes/circuitbox/landing/operational_data/customers',
  'json',
  map('cloudFiles.inferColumnTypes', 'true')
)

![bronze_to_silver.png](attachment:bronze_to_silver.png)

### Move data from bronze to silver using expectations

In [0]:
-- LIVE :Instead of mentioning schema name we can write LIVE which signifies that the table we are refering to in the current pipeline. Only works if the table is in the current pipeline

-- STREAM : If we use the STREAM keyword it only reads the new data from the table. If STREAM is not used, it reads all the data from the table.

CREATE OR REFRESH STREAMING TABLE silver_customers_clean (
  CONSTRAINT valid_customer_id EXPECT (customer_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_customer_name EXPECT (customer_name IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT vaild_telephone EXPECT (LENGTH(telephone) >= 10),
  CONSTRAINT vaild_email EXPECT (email is NOT NULL),
  CONSTRAINT valid_date_of_birth EXPECT(date_of_birth >= '1920-01-01')
)
COMMENT 'Cleaned customers data'
TBLPROPERTIES ('quality' = 'silver')
AS
SELECT
  customer_id,
  customer_name,
  CAST(date_of_birth AS DATE) AS date_of_birth,
  email,
  telephone,
  CAST(created_date AS DATE) AS created_date
FROM STREAM(LIVE.bronze_customers)

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_customers
COMMENT 'SCD Type 1 customers data'
TBLPROPERTIES ('quality' = 'silver')


In [0]:
APPLY CHANGES INTO LIVE.silver_customers
FROM STREAM(LIVE.silver_customers_clean)
KEYS(customer_id)
SEQUENCE BY created_date
STORED AS SCD TYPE 1 --Optional. Type 1 is the default value.
